In [ ]:

!wget "https://huggingface.co/NagisaNao/ANXETY/resolve/main/ComfyUI.zip?download=true" -O ComfyUI.zip
def downloadModel(url, rename=None, token=None):
    if 'huggingface.co' in url:
        if rename:
            filename = rename
        else:
            filename = url.split('/')[-1].split('?')[0]
        print(f"Downloading Hugging Face model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}

    elif 'civitai.com' in url:
        filename = rename or url.split('/')[-1].split('?')[0]
        url_with_token = f"{url}?token={token}" if token else url
        print(f"Downloading Civitai model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url_with_token} -o {filename}
    else:
        print("Unsupported URL format.")

!unzip ComfyUI.zip -d ComfyUI

!mkdir -p /content/ComfyUI/custom_nodes
def install_comfyui_impact():
    %cd /content/ComfyUI/custom_nodes


    !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git


    %cd ComfyUI-Impact-Pack

    !pip install -r requirements.txt

    !pip install ultralytics

    !python install.py
install_comfyui_impact()
!apt-get -y install -qq aria2

%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux
!git clone https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet
!git clone https://github.com/hayden-fr/ComfyUI-Model-Manager
!git clone https://github.com/jags111/efficiency-nodes-comfyui
!git clone https://github.com/XLabs-AI/x-flux-comfyui
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Subpack
!git clone https://github.com/ltdrdata/ComfyUI-Manager
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts
!git clone https://github.com/pythongosssss/ComfyUI-WD14-Tagger
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale
!git clone https://github.com/WASasquatch/was-node-suite-comfyui
!git clone https://github.com/Extraltodeus/LoadLoraWithTags
!git clone https://github.com/liuqianhonga/ComfyUI-Model-Downloader
!git clone https://github.com/X-T-E-R/ComfyUI-EasyCivitai-XTNodes
!git clone https://github.com/crystian/ComfyUI-Crystools
!git clone https://github.com/civitai/civitai_comfy_nodes
!git clone https://github.com/rgthree/rgthree-comfy
!git clone https://github.com/EdithForsaken/ultimate_sd_upscale /content/ComfyUI/custom_nodes/ComfyUI_UltimateSDUpscale/repositories/ultimate_sd_upscale
!git clone https://github.com/ciri/comfyui-model-downloader /content/ComfyUI/custom_nodes/comfyui-downloader
%cd /content/ComfyUI/models/checkpoints
!curl -L -o /content/ComfyUI/models/checkpoints/animagine-xl-4.0-opt.safetensors "https://huggingface.co/cagliostrolab/animagine-xl-4.0/resolve/main/animagine-xl-4.0-opt.safetensors?download=true"
#downloadModel('https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0-pruned.safetensors')
%cd /content/ComfyUI
!wget -O /content/ComfyUI/install-deps.py "https://raw.githubusercontent.com/anxety-solo/sdAIgen/main/__configs__/ComfyUI/install-deps.py"
!wget -O /content/ComfyUI/user/default/comfy.settings.json "https://raw.githubusercontent.com/anxety-solo/sdAIgen/main/__configs__/ComfyUI/comfy.settings.json"
!wget -O /content/ComfyUI/user/default/workflows/anxety-workflow.json "https://raw.githubusercontent.com/anxety-solo/sdAIgen/main/__configs__/ComfyUI/workflows/anxety-workflow.json"





!apt-get -y install -qq aria2

%cd /content/ComfyUI

%run /content/ComfyUI/install-deps.py
%cd /content/ComfyUI
!pip install -r requirements.txt
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --preview-method auto --use-pytorch-cross-attention